In [ ]:
import cvxpy as cp
import numpy as np
import pandas as pd
import json
import datetime
import matplotlib.pyplot as plt
import os
from matplotlib.patches import Patch
parent_dir = os.path.dirname(os.getcwd())
path = parent_dir 
save_path = parent_dir + '/'
from optimization import *
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
color_palette = ['#332288',  '#117733', '#44AA99', '#88CCEE', '#DDCC77', '#CC6677', '#AA4499', '#882255']

## Figure 7: NPC by cluster

In [ ]:
period_string = '2024-06-03_to_2025-06-01'

num_clusters = 15
cluster_labels = np.array(pd.read_csv('Data/Cluster_Results_Adaptive/cluster_labels_'+str(num_clusters)+'.csv', header=None).iloc[:,0])
vinids = np.array(pd.read_csv('Data/vinids.csv').iloc[:,1])

#make a dataframe of vinids and cluster labels
vinids_df = pd.DataFrame({'vinid': vinids, 'cluster': cluster_labels})
label_list = np.unique(vinids_df['cluster'])

In [ ]:
def get_box_data(results_path, period_string):    
# load results into matrices
    discount_rate = 0.045
    years = 100
    low_cc = 6440 + 800 + 5505 # quasar + interconnection fee + install
    medium_cc = 6440 + 800 + 5505 + 3895# quasar + interconnection fee + install + energy management system
    high_cc = 6440 + 800 + (5505 + 3895) * 1.25 # quasar + interconnection fee + install + energy management system * 1.2 for fee in expensive area

    vinids_df = pd.DataFrame({'vinid': vinids, 'cluster': cluster_labels})
    vin_length = 749
    cost_mat_uncontrolled = []
    cost_mat_controlled = []
    cost_mat_v2g_home = []
    cost_mat_v2g_everywhere = []
    vin_not_found_idx = []

    for vin in np.arange(1,vin_length):
        try:
            cost_mat_uncontrolled.append( pd.read_csv(results_path + 'cost_uncontrolled_' + period_string + '_vin_' +str(vin) + '.csv').iloc[:,1])
            cost_mat_controlled.append( pd.read_csv(results_path + 'cost_no_v2g_' + period_string + '_vin_' +str(vin) + '.csv').iloc[:,1])
            cost_mat_v2g_home.append( pd.read_csv(results_path + 'cost_v2g_home_' + period_string + '_vin_' +str(vin) + '.csv').iloc[:,1])
            cost_mat_v2g_everywhere.append(pd.read_csv(results_path + 'cost_v2g_everywhere_' + period_string + '_vin_' +str(vin) + '.csv').iloc[:,1])
        except:
            vin_not_found_idx.append(np.where(vinids==vin)[0])
    
    cost_mat_uncontrolled = np.array(cost_mat_uncontrolled)
    cost_mat_controlled = np.array(cost_mat_controlled)
    cost_mat_v2g_home = np.array(cost_mat_v2g_home)
    cost_mat_v2g_everywhere = np.array(cost_mat_v2g_everywhere)

    for remove in vin_not_found_idx:
        try:
            vinids_df = vinids_df.drop(vinids_df[vinids_df['vinid'] == vinids[remove[0]]].index)
        except:
            pass
    vinids_df = vinids_df.reset_index(drop=True)

    #replace nan with previous value
    for vin in np.arange(cost_mat_controlled.shape[0]):
        for week in np.arange(cost_mat_controlled.shape[1]):
            if np.isnan(cost_mat_controlled[vin, week]):
                if week == 0:
                    cost_mat_controlled[vin, week] = 0
                else:
                    cost_mat_controlled[vin, week] = cost_mat_controlled[vin, week-1]
            if np.isnan(cost_mat_uncontrolled[vin, week]):
                if week == 0:
                    cost_mat_uncontrolled[vin, week] = 0
                else:
                    cost_mat_uncontrolled[vin, week] = cost_mat_uncontrolled[vin, week-1]
            if np.isnan(cost_mat_v2g_home[vin, week]):
                if week == 0:
                    cost_mat_v2g_home[vin, week] = 0
                else:
                    cost_mat_v2g_home[vin, week] = cost_mat_v2g_home[vin, week-1]
            if np.isnan(cost_mat_v2g_everywhere[vin, week]):
                if week == 0:
                    cost_mat_v2g_everywhere[vin, week] = 0
                else:
                    cost_mat_v2g_everywhere[vin, week] = cost_mat_v2g_everywhere[vin, week-1]

    #delete rows that sum up to zero from controlled
    zero_idx = np.all(cost_mat_controlled == 0, axis=1)
    cost_mat_controlled = cost_mat_controlled[~zero_idx]
    cost_mat_uncontrolled = cost_mat_uncontrolled[~zero_idx]
    cost_mat_v2g_home = cost_mat_v2g_home[~zero_idx]
    cost_mat_v2g_everywhere = cost_mat_v2g_everywhere[~zero_idx]

    # drop the rows where zero_idx is True from vinids_df
    vinids_df = vinids_df[~zero_idx]
    vinids_df = vinids_df.reset_index(drop=True)

    num_weeks = cost_mat_uncontrolled.shape[1]
    npc_uncontrolled = np.zeros((cost_mat_uncontrolled.shape[0], num_weeks * years))
    npc_controlled = np.zeros((cost_mat_controlled.shape[0],  num_weeks * years))
    npc_v2g_low = np.zeros((cost_mat_v2g_home.shape[0],  num_weeks * years))
    npc_v2g_medium = np.zeros((cost_mat_v2g_home.shape[0],  num_weeks * years))
    npc_v2g_high = np.zeros((cost_mat_v2g_home.shape[0],  num_weeks * years))

    for year in range(years):
        for week in range(52):
            if year == 0 and week == 0:
                npc_v2g_low[:,0] = low_cc + cost_mat_v2g_home[:,0]
                npc_v2g_medium[:,0] = medium_cc + cost_mat_v2g_home[:,0]
                npc_v2g_high[:,0] = high_cc + cost_mat_v2g_home[:,0]
            else:
                npc_uncontrolled[:, year * 52 + week] = npc_uncontrolled[:, year * 52 + week-1] + cost_mat_uncontrolled[:, week] / np.power(1 + discount_rate, (year * 52+week)/52)
                npc_controlled[:, year * 52 + week] = npc_controlled[:, year * 52 + week-1] + cost_mat_controlled[:, week] / np.power(1 + discount_rate, (year * 52+week)/52)
                npc_v2g_low[:, year * 52 + week] = npc_v2g_low[:, year * 52 + week-1] + cost_mat_v2g_home[:, week] / np.power(1 + discount_rate, (year * 52+week)/52)
                npc_v2g_medium[:, year * 52 + week] = npc_v2g_medium[:, year * 52 + week-1] + cost_mat_v2g_home[:, week] / np.power(1 + discount_rate, (year * 52+week)/52)
                npc_v2g_high[:, year * 52 + week] = npc_v2g_high[:, year * 52 + week-1] + cost_mat_v2g_home[:, week] / np.power(1 + discount_rate, (year * 52+week)/52)

    #find average breakeven week averaging all vehicles for low, medium and high v2g cost
    breakeven_week_low = np.where(np.nanmean(npc_uncontrolled, axis=0) > np.nanmean(npc_v2g_low, axis=0))[0]
    if breakeven_week_low.size > 0:
        breakeven_week_low = breakeven_week_low[0]
    else:
        breakeven_week_low = np.nan
    breakeven_week_medium = np.where(np.mean(npc_uncontrolled, axis=0) > np.mean(npc_v2g_medium, axis=0))[0]
    if breakeven_week_medium.size > 0:
        breakeven_week_medium = breakeven_week_medium[0]
    else:
        breakeven_week_medium = np.nan
    breakeven_week_high = np.where(np.mean(npc_uncontrolled, axis=0) > np.mean(npc_v2g_high, axis=0))[0]
    if breakeven_week_high.size > 0:    
        breakeven_week_high = breakeven_week_high[0]
    else:
        breakeven_week_high = np.nan

    #find breakeven week for each vehicle for low, medium and high v2g cost
    for vin in np.arange(cost_mat_uncontrolled.shape[0]):
        breakeven_idx = np.where(npc_uncontrolled[vin,:] > npc_v2g_low[vin,:])[0]
        if breakeven_idx.size > 0:
            breakeven_week = breakeven_idx[0]
        else:
            breakeven_week = np.nan
        vinids_df.loc[vin, 'breakeven_week_low'] = breakeven_week

        breakeven_idx = np.where(npc_uncontrolled[vin,:] > npc_v2g_medium[vin,:])[0]
        if breakeven_idx.size > 0:
            breakeven_week = breakeven_idx[0]
        else:
            breakeven_week = np.nan
        vinids_df.loc[vin, 'breakeven_week_medium'] = breakeven_week    
        breakeven_idx = np.where(npc_uncontrolled[vin,:] > npc_v2g_high[vin,:])[0]
        if breakeven_idx.size > 0:
            breakeven_week = breakeven_idx[0]
        else:
            breakeven_week = np.nan
        vinids_df.loc[vin, 'breakeven_week_high'] = breakeven_week
    return vinids_df, breakeven_week_low, breakeven_week_medium, breakeven_week_high

In [ ]:
def plot_box(ax, box_data_low, box_data_medium, box_data_high, y_max=15):
    color_dark = "#437A98" #70A3BF'
    color_light = "#BFE5F6"
    
    ax.boxplot(box_data_low, positions=np.arange(1,num_clusters+1)-0.22, widths=0.2, patch_artist=True, boxprops=dict(facecolor=color_light, color=color_light), medianprops=dict(color='black'), whiskerprops=dict(color='black'), capprops=dict(color='black'), showfliers=False, flierprops=dict(markerfacecolor=color_light, marker='o'))
    ax.boxplot(box_data_medium, positions=np.arange(1,num_clusters+1), widths=0.2, patch_artist=True, boxprops=dict(facecolor=color_palette[3], color=color_palette[3]), medianprops=dict(color='black'), whiskerprops=dict(color='black'), capprops=dict(color='black'), showfliers=False, flierprops=dict(markerfacecolor=color_palette[3], marker='o'))
    ax.boxplot(box_data_high, positions=np.arange(1,num_clusters+1)+ 0.22, widths=0.2, patch_artist=True, boxprops=dict(facecolor=color_dark, color=color_dark), medianprops=dict(color='black'), whiskerprops=dict(color='black'), capprops=dict(color='black'), showfliers=False, flierprops=dict(markerfacecolor=color_dark, marker='o'))

    ax.set_ylabel('Break-even Time [years]', fontsize=16)
    ax.set_xlabel('Cluster', fontsize=16)
    ax.set_xticks(np.arange(1,num_clusters+1))
    ax.set_xticklabels(np.arange(1,num_clusters+1), fontsize=14) 
    ax.set_ylim(0, y_max)
    ax.tick_params(axis='both', which='major', labelsize=14)

In [ ]:
fig, ax = plt.subplots(4,1,figsize=(13, 18))
plt.subplots_adjust(hspace=0.3)
plt.subplots_adjust(wspace=0.15)
color_dark = "#437A98" #70A3BF'
color_light = "#BFE5F6"

ax = ax.flatten()
cluster_info_df = pd.read_csv('Data/Cluster_Results_Adaptive/cluster_info_'+str(num_clusters)+'.csv')
vinids_dict = {}
y_max = [20, 20, 30, 50]

for i, circuit in enumerate(['cir_1', 'cir_2', 'cir_3', 'cir_4']):
    aging = 'batt_aging_0'
    results_path = 'Results/'+ circuit + '/' + aging + '/' + 'elrp_1_'
    tmp, break_low, break_med, break_high = get_box_data( results_path, period_string)
    vinids_dict[f"string{i}"] = tmp
    #print average breakeven time for low, medium, and high capital costs
    print('Circuit '+str(i+1) + ' breakeven times:')
    print(np.round(break_low/52, 2), ' & ', np.round(break_med/52, 2), ' & ', np.round(break_high/52,2))

    box_data_v2g_low = []
    box_data_v2g_medium = []
    box_data_v2g_high = []
    for c in range(num_clusters): 
        c_label = cluster_info_df['Orig Cluster Label'].values[c]
        tmp_low = vinids_dict[f"string{i}"].loc[vinids_dict[f"string{i}"].cluster == c_label,'breakeven_week_low']/52
        tmp_medium = vinids_dict[f"string{i}"].loc[vinids_dict[f"string{i}"].cluster == c_label,'breakeven_week_medium']/52
        tmp_high = vinids_dict[f"string{i}"].loc[vinids_dict[f"string{i}"].cluster == c_label,'breakeven_week_high']/52

        box_data_v2g_low.append(tmp_low[~np.isnan(tmp_low)])
        box_data_v2g_medium.append(tmp_medium[~np.isnan(tmp_medium)])
        box_data_v2g_high.append(tmp_high[~np.isnan(tmp_high)])
    plot_box(ax[i], box_data_v2g_low, box_data_v2g_medium, box_data_v2g_high, y_max=y_max[i])
    ax[i].annotate('Circuit ' + str(i+1), xy=(0.9, .92), xycoords='axes fraction', ha='center', fontsize=16)
    legend_elements = [Patch(facecolor=color_light, color=color_light,
                                label= 'Low Cap. Cost'),
                        Patch(facecolor=color_palette[3], color=color_palette[3],
                                label= 'Medium Cap. Cost'),
                        Patch(facecolor=color_dark, color=color_dark,
                                label= 'High Cap. Cost')]
    ax[i].legend(handles=legend_elements,  fontsize=14)

ax[0].text(-0.1, 1.05, 'a.', transform 
            = ax[0].transAxes, fontsize=18)
ax[1].text(-0.1, 1.05, 'b.', transform 
            = ax[1].transAxes, fontsize=18)
ax[2].text(-0.1, 1.05, 'c.', transform 
            = ax[2].transAxes, fontsize=18)
ax[3].text(-0.1, 1.05, 'd.', transform 
            = ax[3].transAxes, fontsize=18)

#print table with average breakeven times by cluster for each circuit
vinids_df_1 = vinids_dict[f"string{0}"]
vinids_df_2 = vinids_dict[f"string{1}"]
vinids_df_3 = vinids_dict[f"string{2}"]
vinids_df_4 = vinids_dict[f"string{3}"]
print('Average breakeven times by cluster:')
print('Cluster, Circuit 1, Circuit 2, Circuit 3, Circuit 4')
for c in range(num_clusters): 
    c_label = cluster_info_df['Orig Cluster Label'].values[c]  
    print('   & Low & ' + str(np.round(np.nanmean(vinids_df_1.loc[vinids_df_1.cluster == c_label, 'breakeven_week_low'])/52, 2)) + ' & ' + str(np.round(np.nanmean(vinids_df_2.loc[vinids_df_2.cluster == c_label, 'breakeven_week_low'])/52, 2)) +   ' & ' + str(np.round(np.nanmean(vinids_df_3.loc[vinids_df_3.cluster == c_label, 'breakeven_week_low'])/52, 2)) +' & ' + str(np.round(np.nanmean(vinids_df_4.loc[vinids_df_4.cluster == c_label, 'breakeven_week_low'])/52, 2)) + '\\\\')
    #print same line but for medium and high breakeven times
    print(str(c+1) + ' & Medium & ' + str(np.round(np.nanmean(vinids_df_1.loc[vinids_df_1.cluster == c_label, 'breakeven_week_medium'])/52, 2)) + ' & ' + str(np.round(np.nanmean(vinids_df_2.loc[vinids_df_2.cluster == c_label, 'breakeven_week_medium'])/52, 2)) +   ' & ' + str(np.round(np.nanmean(vinids_df_3.loc[vinids_df_3.cluster == c_label, 'breakeven_week_medium'])/52, 2)) +' & ' + str(np.round(np.nanmean(vinids_df_4.loc[vinids_df_4.cluster == c_label, 'breakeven_week_medium'])/52, 2)) + '\\\\')
    print(  '   & High & ' + str(np.round(np.nanmean(vinids_df_1.loc[vinids_df_1.cluster == c_label, 'breakeven_week_high'])/52, 2)) + ' & ' + str(np.round(np.nanmean(vinids_df_2.loc[vinids_df_2.cluster == c_label, 'breakeven_week_high'])/52, 2)) +   ' & ' + str(np.round(np.nanmean(vinids_df_3.loc[vinids_df_3.cluster == c_label, 'breakeven_week_high'])/52, 2)) +' & ' + str(np.round(np.nanmean(vinids_df_4.loc[vinids_df_4.cluster == c_label, 'breakeven_week_high'])/52, 2)) + '\\\\')
    print('\midrule')
plt.savefig(save_path + '7_npc_clusters_box.pdf', bbox_inches='tight')
